# code to discretize the CASH+ solid oslution model into pure phases of given composition

In [1]:
# importing necessary functions
from discretizer.cashplus import discretize_cash, add_cash_to_fundb, generate_reactions_one_by_one

In [2]:
# defining discretization; composition interval and steps
discretization = [{'ratio': 'Ca/Si','interval': [0.7, 1.65], 'step': 0.2}, 
                  {'ratio': 'Na/Si','interval': [0.0, 0.3], 'step': 0.1 }, 
                  {'ratio': 'Al/Si','interval': [0.0, 0.1], 'step': 0.1 }
                 ]

In [3]:
# discretizeing model calling GEMS to calculate the real solution properties for each composition
dicretized_cash = discretize_cash(discretization)

For Ca/Si 0.7 Na/Si 0.0 Al/Si 0.0 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(H2O)1.0391
For Ca/Si 0.7 Na/Si 0.0 Al/Si 0.1 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Al2O3)0.025(H2O)1.125
For Ca/Si 0.7 Na/Si 0.1 Al/Si 0.0 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Na2O)0.05(H2O)1.0963
For Ca/Si 0.7 Na/Si 0.1 Al/Si 0.1 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Na2O)0.05(Al2O3)0.025(H2O)1.175
For Ca/Si 0.7 Na/Si 0.2 Al/Si 0.0 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Na2O)0.1(H2O)1.1499
For Ca/Si 0.7 Na/Si 0.2 Al/Si 0.1 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Na2O)0.1(Al2O3)0.025(H2O)1.225
For Ca/Si 0.7 Na/Si 0.3 Al/Si 0.0 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Na2O)0.1498(H2O)1.1998
For Ca/Si 0.7 Na/Si 0.3 Al/Si 0.1 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Na2O)0.15(Al2O3)0.025(H2O)1.275
For Ca/Si 0.9 Na/Si 0.0 Al/Si 0.0 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.

In [4]:
# adding discrete phases to cemdata database 
import thermohubclient as client
import thermofun as fun
dbc = client.DatabaseClient()

dbc.saveDatabase('cemdata18')

fun_cemdata18 = fun.Database('cemdata18-thermofun.json')

In [5]:
fun_cemdata18 = add_cash_to_fundb(fun_cemdata18, dicretized_cash)

In [6]:
# generating chemical reqaction for new phases, with provided master species
aqueous_master_species = ['H2O@', 'Na+', 'K+', 'Al+3', 'SiO2@', 'Ca+2', 'H+', 'HCO3-']
reactions_dic, reactions_list = generate_reactions_one_by_one(aqueous_master_species, [c['formula'] for c in dicretized_cash] )

0.7Ca+2 + SiO2@ + 1.7391H2O@ =  + 1.4H+
0.7Ca+2 + SiO2@ + 0.05Al+3 + 1.9H2O@ =  + 1.55H+
0.7Ca+2 + SiO2@ + 0.1Na+ + 1.8463H2O@ = (SiO2)1.0(CaO)0.7(Na2O)0.05(H2O)1.0963 + 1.5H+
0.7Ca+2 + SiO2@ + 0.05Al+3 + 0.1Na+ + 2H2O@ =  + 1.65H+
0.7Ca+2 + SiO2@ + 0.2Na+ + 1.9499H2O@ =  + 1.6H+
0.7Ca+2 + SiO2@ + 0.05Al+3 + 0.2Na+ + 2.1H2O@ =  + 1.75H+
0.7Ca+2 + SiO2@ + 0.2996Na+ + 2.0496H2O@ =  + 1.6996H+
0.7Ca+2 + SiO2@ + 0.05Al+3 + 0.3Na+ + 2.2H2O@ =  + 1.85H+
0.9Ca+2 + SiO2@ + 2.1676H2O@ =  + 1.8H+
0.9Ca+2 + SiO2@ + 0.1Al+3 + 2.4776H2O@ =  + 2.1H+
0.9Ca+2 + SiO2@ + 0.1Na+ + 2.2818H2O@ =  + 1.9H+
0.9Ca+2 + SiO2@ + 0.1Al+3 + 0.1Na+ + 2.5902H2O@ =  + 2.2H+
0.9Ca+2 + SiO2@ + 0.2Na+ + 2.3933H2O@ = (SiO2)1.0(CaO)0.9(Na2O)0.1(H2O)1.3933 + 2H+
0.9Ca+2 + SiO2@ + 0.1Al+3 + 0.2Na+ + 2.7032H2O@ = (SiO2)1.0(CaO)0.9(Na2O)0.1(Al2O3)0.05(H2O)1.5532 + 2.3H+
0.9Ca+2 + SiO2@ + 0.2998Na+ + 2.5002H2O@ =  + 2.0998H+
0.9Ca+2 + SiO2@ + 0.1Al+3 + 0.2998Na+ + 2.8133H2O@ = (SiO2)1.0(CaO)0.9(Na2O)0.1499(Al2O3)0.05(H2O)1.6134

In [9]:
# calculating reaction properties for given T list
T_list = [0,25,60,100,150,200,250, 300]
engine = fun.ThermoEngine(fun_cemdata18) # thermofun engine is used to calculate the properties of reactions
# collecting results from the loop
reactions_logKs =[ [engine.thermoPropertiesReaction(t+273.15001, 0, r).log_equilibrium_constant.val for t in T_list] for r in reactions_list ] 

In [10]:
# exporting to CHESS database format
fun_substances = [fun_cemdata18.getSubstance(c['formula'])for c in dicretized_cash]

from datetime import date
today = date.today()

file_name = today.strftime("%b-%d-%Y")+ '_chess_discrete_data.txt'

from common.export import to_chess
to_chess(file_name, fun_substances, reactions_dic, T_list, reactions_logKs, 'cash+ valid to 100C')